**Neural Network - Machine 98**

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pandas as pd
import tensorflow as tf
import numpy as np

In [3]:
# Import our input dataset
machine_98_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/Resources/machine_98.csv')
machine_98_df = machine_98_df.drop_duplicates(subset=['datetime'], keep='first').reset_index(drop=True)
machine_98_df.head()

,datetime,machineid,machinemodel,machineage,volt,rotate,pressure,vibration,errorid,compmaint,compfail
0,2015-04-30 06:00:00,98,model2,20,154.125119,343.460135,85.853673,46.107801,0,1,1
1,2015-06-14 06:00:00,98,model2,20,188.339367,383.655684,87.714489,41.516273,0,1,1
2,2015-06-29 06:00:00,98,model2,20,231.317520,389.259020,90.300412,50.545201,0,1,1
3,2015-11-11 06:00:00,98,model2,20,177.206367,423.824696,102.044302,38.968430,0,1,0
4,2015-09-12 06:00:00,98,model2,20,185.221725,464.308972,109.791304,40.912735,0,1,0


In [4]:
#check for nulls
machine_98_df.count()

datetime        8761
machineid       8761
machinemodel    8761
machineage      8761
volt            8761
rotate          8761
pressure        8761
vibration       8761
errorid         8761
compmaint       8761
compfail        8761
dtype: int64

In [5]:
#remove unecessary columns
machine_98_df = machine_98_df.drop(columns=['machineid', 'machinemodel', 'machineage', 'datetime'])
machine_98_df.head()

,volt,rotate,pressure,vibration,errorid,compmaint,compfail
0,154.125119,343.460135,85.853673,46.107801,0,1,1
1,188.339367,383.655684,87.714489,41.516273,0,1,1
2,231.317520,389.259020,90.300412,50.545201,0,1,1
3,177.206367,423.824696,102.044302,38.968430,0,1,0
4,185.221725,464.308972,109.791304,40.912735,0,1,0


In [ ]:
# split preprocessed data into feature and target arrays
y = machine_98_df.compfail.values
X = machine_98_df.drop(columns='compfail').values

# split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 35
hidden_nodes_layer2 = 15

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 35)                245       
                                                                 
 dense_1 (Dense)             (None, 15)                540       
                                                                 
 dense_2 (Dense)             (None, 1)                 16        
                                                                 
Total params: 801 (3.13 KB)
Trainable params: 801 (3.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, validation_data=(X_test, y_test), epochs=100)

Epoch 1/100
206/206 [==============================] - 1s 2ms/step - loss: 1.9354e-04 - accuracy: 1.0000 - val_loss: 1.8227 - val_accuracy: 0.9982
Epoch 2/100
206/206 [==============================] - 0s 2ms/step - loss: 4.0404e-04 - accuracy: 0.9998 - val_loss: 1.9686 - val_accuracy: 0.9982
Epoch 3/100
206/206 [==============================] - 0s 2ms/step - loss: 1.0169e-04 - accuracy: 1.0000 - val_loss: 2.1581 - val_accuracy: 0.9982
Epoch 4/100
206/206 [==============================] - 0s 2ms/step - loss: 2.0759e-04 - accuracy: 1.0000 - val_loss: 2.1478 - val_accuracy: 0.9982
Epoch 5/100
206/206 [==============================] - 0s 2ms/step - loss: 1.2271e-04 - accuracy: 1.0000 - val_loss: 2.0729 - val_accuracy: 0.9982
Epoch 6/100
206/206 [==============================] - 0s 2ms/step - loss: 1.0442e-04 - accuracy: 1.0000 - val_loss: 2.0455 - val_accuracy: 0.9982
Epoch 7/100
206/206 [==============================] - 0s 2ms/step - loss: 6.2970e-05 - accuracy: 1.0000 - val_loss: 2

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


69/69 - 0s - loss: 0.0156 - accuracy: 0.9982 - 73ms/epoch - 1ms/step
Loss: 0.01564565859735012, Accuracy: 0.9981743693351746


In [ ]:
# Making predictions using the testing data
predictions = nn.predict(X_test_scaled)
predictions_bool = np.argmax(predictions, axis=1)

# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions_bool)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions_bool)

# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions_bool))

69/69 [==============================] - 0s 943us/step
Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,2187,0
Actual 1,4,0


Accuracy Score : 0.9981743496120493
Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2187
           1       0.00      0.00      0.00         4

    accuracy                           1.00      2191
   macro avg       0.50      0.50      0.50      2191
weighted avg       1.00      1.00      1.00      2191



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
